In [1]:
# 체 Field
'''
사칙 연산이 정의되는 집합
원소의 개수가 유한개인 체 := 유한체
'''

In [2]:
# 유한체 (Finite Field)
p = 7
F_p = [i for i in range(p)]

In [3]:
# 모듈러 연산(== 나머지 연산) 
def Modular(a:int, b:int, cal_op:['+', '-','/', '*'], mod:int):
    if cal_op == '+':
        result = (a + b) % mod
    elif cal_op == '-':
        result = (a - b) % mod
    elif cal_op == '*':
        result = (a * b) % mod
    elif cal_op == '/':
        for i in range(mod):
            if (b * i) % mod == 1:
                b = i
                break        
        result = (a * b) % mod
    else:
        raise Exception("연산기호가 맞지 않습니다. : '+', '-','/', '*' ")
        
    return result

In [11]:
Modular(2, 3, '-', 7)

6

In [ ]:
##################################### Elliptic curve Modular Arithmethic ################################

In [5]:
from typing import *

In [7]:
def EllipticModular(P1:List, P2:List, cal_op:['neg', 'add','mul'], mod:int, a, b):
    """
    P1, P2 := (x, y)
    cal_op := ['neg', 'add','mul']
        neg == negation 
            neg P = -P
        add == addition 
            P1 + P2 = R
        mul == scalar multiplication
            scalr * P1
    mod := int
    y ** 2 = x ** 3 + a*x + b
    4 * (a**3) + 27 * (b ** 2) != 0
    """
    
    assert (isinstance(P1, List)) \
            and (isinstance(P2, List) or P2 is None), "P1 must be List and P2 must be List or None" 
    
    assert 4 * (a**3) + 27 * (b ** 2) != 0, 'Avoid Crossing in Elliptic!'
    
    if cal_op == 'add':
        s = Modular((P2[1] - P1[1]), (P2[0] - P1[0]), '/', mod)
        xr = Modular(s ** 2 - P1[0], P2[0], '-', mod)
        yr = Modular(s * (P1[0] - xr), P1[1], '-', mod)
        result = (xr, yr)

    elif cal_op == 'mul':
        scalar = int(input("Scalar : "))
        assert isinstance(scalar, int), "scalar must be positive int !"
        if scalar == 2:
            s = Modular((3*(P1[0] ** 2) + a), (2 * P1[1]), '/', mod)
            xr = Modular(s ** 2, 2*P1[0], '-', mod)
            yr = Modular(s * (P1[0] - xr), P1[1], '-', mod)
            result = (xr, yr)
        else:
            s = Modular((3*(P1[0] ** 2) + a), (2 * P1[1]), '/', mod)
            xr = Modular(s ** 2, 2*P1[0], '-', mod)
            yr = Modular(s * (P1[0] - xr), P1[1], '-', mod)
            P_ = [xr, yr]
            for i in range(1, scalar - 1):
                s = Modular(P1[1] - P_[1], P1[0] - P_[0], '/', mod)
                xr = Modular(s ** 2 - P1[0], P_[0], '-', mod)
                yr = Modular(s * (P1[0] - xr), P1[1], '-', mod)
                P_ = [xr, yr]
            result = (P_[0], P_[1])
    else:
        raise Exception("연산기호가 맞지 않습니다. : ['neg', 'add','mul'] ")
        
    return result

In [9]:
EllipticModular(P1=[15, 13], P2=None, cal_op='mul', mod=17, a=0, b=7)

Scalar : 4


(12, 1)

In [118]:
EllipticModular(P1=[15, 13], P2=[12, 1], cal_op='add', mod=17, a=0, b=7)

(6, 6)

In [ ]:
#############################  OPEN SOURCE 에서 가져온 파일  ##########################

In [12]:
#####################Private key ---------------->Public key ######################

In [1]:
from hashlib import sha256
from typing import Union


class Point(object):
    def __init__(self, _x, _y, _order=None):
        self.x, self.y, self.order = _x, _y, _order

    def calc(self, top, bottom, other_x):
        ll = (top * inverse_mod(bottom)) % p
        x3 = (ll * ll - self.x - other_x) % p
        return Point(x3, (ll * (self.x - x3) - self.y) % p)

    def double(self):
        if self == INFINITY: return INFINITY
        return self.calc(3 * self.x * self.x, 2 * self.y, self.x)

    def __add__(self, other):
        if other == INFINITY:
            return self
        if self == INFINITY:
            return other
        if self.x == other.x:
            if (self.y + other.y) % p == 0:
                return INFINITY
            return self.double()
        return self.calc(other.y - self.y, other.x - self.x, other.x)

    def __mul__(self, e):
        if self.order:
            e %= self.order
        if e == 0 or self == INFINITY:
            return INFINITY
        result, q = INFINITY, self
        while e:
            if e & 1:
                result += q
            e, q = e >> 1, q.double()
        return result

    def __str__(self):
        if self == INFINITY:
            return "infinity"
        return "%x %x" % (self.x, self.y)


p, INFINITY = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F, Point(None, None)  # secp256k1
g = Point(0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798,
          0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8,
          0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141)


def inverse_mod(a):
    if a < 0 or a >= p:
        a = a % p
    c, d, uc, vc, ud, vd = a, p, 1, 0, 0, 1
    while c:
        q, c, d = divmod(d, c) + (c,)
        uc, vc, ud, vd = ud - q * uc, vd - q * vc, uc, vc
    if ud > 0:
        return ud
    return ud + p


# Thanks for this snippet to David Keijser
# https://github.com/keis/base58/

BITCOIN_ALPHABET = b'123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'


def scrub_input(v: Union[str, bytes]) -> bytes:
    if isinstance(v, str):
        v = v.encode('ascii')

    return v


def b58encode_int(
        i: int, default_one: bool = True, alphabet: bytes = BITCOIN_ALPHABET
) -> bytes:
    """
    Encode an integer using Base58
    """
    if not i and default_one:
        return alphabet[0:1]
    string = b""
    while i:
        i, idx = divmod(i, 58)
        string = alphabet[idx:idx + 1] + string
    return string


def b58encode(v: Union[str, bytes], alphabet: bytes = BITCOIN_ALPHABET) -> bytes:
    """
    Encode a string using Base58
    """
    v = scrub_input(v)

    nPad = len(v)
    v = v.lstrip(b'\0')
    nPad -= len(v)

    p, acc = 1, 0
    for c in reversed(v):
        acc += p * c
        p = p << 8
    result = b58encode_int(acc, default_one=False, alphabet=alphabet)
    return alphabet[0:1] * nPad + result


def b58decode_int(v: Union[str, bytes], alphabet: bytes = BITCOIN_ALPHABET) -> int:
    """
    Decode a Base58 encoded string as an integer
    """
    v = v.rstrip()
    v = scrub_input(v)

    decimal = 0
    for char in v:
        decimal = decimal * 58 + alphabet.index(char)
    return decimal


def b58decode(v: Union[str, bytes], alphabet: bytes = BITCOIN_ALPHABET) -> bytes:
    """
    Decode a Base58 encoded string
    """
    v = v.rstrip()
    v = scrub_input(v)

    origlen = len(v)
    v = v.lstrip(alphabet[0:1])
    newlen = len(v)

    acc = b58decode_int(v, alphabet=alphabet)

    result = []
    while acc > 0:
        acc, mod = divmod(acc, 256)
        result.append(mod)

    return b'\0' * (origlen - newlen) + bytes(reversed(result))


In [3]:
import argparse
import hashlib

from binascii import hexlify, unhexlify
from struct import Struct

PACKER = Struct('>QQQQ')


def count_leading_zeroes(s):
    count = 0
    for c in s:
        if c == '\0':
            count += 1
        else:
            break
    return count


def base58_check_encode(prefix, payload, compressed=False):
    # Add version byte in front of RIPEMD-160 hash (0x00 for Main Network)
    s = prefix + payload
    if compressed:
        s = prefix + payload + b'\x01'

    # Add the 4 checksum bytes at the end of extended RIPEMD-160 hash. This is the 25-byte binary Bitcoin Address.
    checksum = hashlib.sha256(hashlib.sha256(s).digest()).digest()[0:4]

    result = s + checksum

    return '1' * count_leading_zeroes(result) + b58encode(result).decode()


def pub_key_to_addr(s):
    ripemd160 = hashlib.new('ripemd160')

    hash_sha256 = hashlib.new('SHA256')

    # Perform SHA-256 hashing on the public key
    hash_sha256.update(bytes.fromhex(s))

    # Perform RIPEMD-160 hashing on the result of SHA-256
    ripemd160.update(hash_sha256.digest())

    return base58_check_encode(b'\0', ripemd160.digest())


def int_to_address(number):
    number0 = number >> 192
    number1 = (number >> 128) & 0xffffffffffffffff
    number2 = (number >> 64) & 0xffffffffffffffff
    number3 = number & 0xffffffffffffffff

    private_key = hexlify(PACKER.pack(number0, number1, number2, number3)).decode("utf-8")

    print('Converting from: ' + str(int(private_key, 16)))

    compressed_key = base58_check_encode(b'\x80', unhexlify(private_key), True)
    print('Private key: ' + compressed_key)

    # address
    x, y = str(g * int(private_key, 16)).split()
    len1 = len(x)
    len2 = len(y)
    if len1 != 64:
        z = 64 - len1
        x = '0' * z + x

    if len2 != 64:
        z = 64 - len2
        y = '0' * z + y
    compressed_public_key_with_out_prefix = x + y

    pk_prefix = '02'
    if not int(compressed_public_key_with_out_prefix[64:], 16) % 2 == 0:
        pk_prefix = '03'
    compressed_public_key = pk_prefix + compressed_public_key_with_out_prefix[:64]

    print('Public key: ' + compressed_public_key)
    print('Bitcoin address: ' + pub_key_to_addr(compressed_public_key))


def wif_to_key(wif):
    slicer = 4
    if wif[0] in ['K', 'L']:
        slicer = 5

    return hexlify(b58decode(wif)[1:-slicer]).decode('utf-8')


def main():
    #parser = argparse.ArgumentParser(description='Generates private key, public key and wallet address from number')

    #parser.add_argument('number', type=int, nargs='?', default=1,
                        #help='A required integer number argument')
    #args = parser.parse_args()
    
    #int_to_address(args.number)
    number = int(input('Private key:'))
    int_to_address(number)


# int_to_address(12345678900987654321)


if __name__ == "__main__":
    main()

# For Donation: 1z4QU2cwJMorUq9ouUSbZNhHhQhCT4wRc


Private key:12048542875
Converting from: 12048542875
Private key: KwDiBf89QgGbjEhKnhXJuH7LrciVrZi3qYjgd9MGzS8HnQ6MNwCP
Public key: 02b7211ecf90346dae2508d697f8096995c0683e00d75ea5d6dac8487ac966771f
Bitcoin address: 15xXyRUY1rhPsu5Ci9nhoTUvaSQKQo3Etx
